<a href="https://colab.research.google.com/github/BankNatchapol/Comparison-Of-Quantum-Gradient/blob/main/concept_implementation/quantum_gate_estimation_lcu.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install pennylane-sf

In [ ]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import GradientDescentOptimizer

import pandas as pd

import matplotlib.pyplot as plt
import plotly.graph_objects as go
import plotly.express as px

In [ ]:
#@title Auxilary function
import warnings
warnings.filterwarnings("ignore")

param_gates = ['RX', 'RY', 'RZ']

def padding(params):
    j = 0
    padded = []
    for i in range(len(ansatz)):
      if ansatz[i] in param_gates:
        padded.append(params[j])
        j+=1
      else:
        padded.append(None)
    return np.array(padded)

def projector1(wires):
  projector = np.zeros((2, 2))
  projector[1,1] = 1
  qml.QubitUnitary(projector, wires=wires)

def str2gate(gate):
  if gate == 'H':
    return qml.Hadamard
  elif gate == 'X':
    return qml.PauliX
  elif gate == 'Y':
    return qml.PauliY
  elif gate == 'Z':
    return qml.PauliZ
  elif gate == 'RX':
    return qml.RX
  elif gate == 'RY':
    return qml.RY
  elif gate == 'RZ':
    return qml.RZ
  elif gate == 'Projector1':
    return projector1
  else:
    assert 0, "Gate not support."

def controlStr(gate):
  if gate == 'RX':
    return qml.CNOT
  elif gate == 'RY':
    return qml.CY
  elif gate == 'RZ':
    return qml.CZ
  else:
    assert 0, "Gate not support."
    
def print_ansatz(ansatz, coeffs, wires):
  coeffs = padding(coeffs)
  @qml.qnode(dev)
  def ansatz_construct(ansatz, coeffs, wires):
    ansatz = [a.numpy() for a in ansatz]
    coeffs = [c.numpy() for c in coeffs]
    wires = [w.numpy() for w in wires]

    for i, coeff in enumerate(coeffs):
      if not coeff:
        if ansatz[i] ==  'Problem':
          problem(wires[i])
        else:
          str2gate(ansatz[i])(wires=wires[i])
      else:
        str2gate(ansatz[i])(coeff, wires=wires[i])

    return qml.state()
  print(qml.draw(ansatz_construct)(ansatz, coeffs, wires))

Creating ansatz

In [ ]:
ansatz = ['H', 'RZ', 'RX']
params_init = np.array([np.pi/4, np.pi])
wires = [0, 0, 0]

print_ansatz(ansatz, params_init, wires)

 0: ──H──RZ(0.785)──RX(3.14)──┤ State 



Hamiltonian $H = | ⟨\psi|1⟩|^2$

In [ ]:
H = ['Projector1']
ham_wires = [0]
ham_coeffs = [1.0]

Gradient checking using Pennylane implemented parameter shift gradient

In [ ]:
num_wires = 1
dev1 = qml.device("default.qubit", wires=num_wires)

num_wires = 2
dev2 = qml.device("default.qubit", wires=num_wires)

In [ ]:
@qml.qnode(dev1)
def ansatz_circuit(params):
  qml.Hadamard(wires=0)
  qml.RZ(params[0], wires=0)
  qml.RX(params[1], wires=0)
  return qml.expval(qml.Projector([1], wires=0))

In [ ]:
gradfn = qml.grad(ansatz_circuit)
grads_param_shift = gradfn(params_init)
print("Target gradients: ", grads_param_shift)

Target gradients:  [-5.55111512e-17  3.53553391e-01]


Construct linear combination of unitaries(LCU) gradients

In [ ]:
def lcu_term(ansatz, params, wires, diff_index):
  qml.Hadamard(wires=1)
  qml.adjoint(qml.S)(wires=1)

  for i, gate in enumerate(ansatz):
    if gate in param_gates:
      if i == diff_index:
        controlStr(gate)(wires=[1, 0])
      str2gate(gate)(params[i], wires=wires[i])
    else:
      if gate == 'Problem':
        problem(wires[i])
      else:
        str2gate(gate)(wires=wires[i])
    
  qml.Hadamard(wires=1)


def lcu_term_adjointed(ansatz, params, wires, diff_index):
  ansatz = [a for a in reversed(ansatz)]
  params = params[::-1]

  qml.Hadamard(wires=1)
  for i, gate in enumerate(ansatz):
    if gate in param_gates:
      str2gate(gate)(-params[i], wires=wires[i])
      if i == len(ansatz)-diff_index-1:
        controlStr(gate)(wires=[1, 0])
    else:
      if gate == 'Problem':
         qml.adjoint(problem)(wires[i])
      else:
        str2gate(gate)(wires=wires[i])

  qml.S(wires=1)
  qml.Hadamard(wires=1)


@qml.qnode(dev2)
def lcu(ansatz, H, ham_wires, params, wires, diff_index):
  ansatz = [a.numpy() for a in ansatz]
  wires = [w.numpy() for w in wires]
  ham_wires = ham_wires.numpy()

  lcu_term(ansatz, params, wires, diff_index)
  str2gate(H)(wires=ham_wires)
  str2gate('Z')(wires=1)
  lcu_term_adjointed(ansatz, params, wires, diff_index)

  return qml.state()

def lcu_gradients(ansatz, H, ham_wires, ham_coeffs, params, wires):

  params = padding(params)

  gradients = []
  for i in range(len(ansatz)):
    if ansatz[i] in param_gates:
      gradients.append(sum([lcu(ansatz, H[h], ham_wires[h], params, wires, i)[0].numpy()*ham_coeffs[h] for h in range(len(H))]).real)

  return gradients

In [ ]:
print(qml.draw(lcu)(ansatz, H[0], ham_wires[0], padding(params), wires, 2))

 0: ──H──RZ(0.0812)──╭X──RX(0.0407)──U0──RX(-0.0407)──╭X──RZ(-0.0812)──H──╭┤ State 
 1: ──H──S⁻¹─────────╰C──H───────────Z───H────────────╰C──S────────────H──╰┤ State 
U0 =
[[0. 0.]
 [0. 1.]]



In [ ]:
ansatz = ['H', 'RZ', 'RX']
wires = [0, 0, 0]

num_wires = 1 
num_layers = 2

np.random.seed(1)
params_init= 0.05*np.random.randn(1, 3*num_wires*num_layers)[0]
params_init

In [ ]:
grads_lcu = lcu_gradients(ansatz, H, ham_wires, ham_coeffs, params_init, wires)

In [ ]:
print("Pennylane implemented gradient : ", [grad for grad in grads_param_shift])
print("LCU gradient                   : ", [grad for grad in grads_lcu])

Pennylane implemented gradient :  [-5.551115123125783e-17, 0.3535533905932737]
LCU gradient                   :  [-7.716005263714514e-17, 0.3535533905932735]


# **Quantum gate estimation using LCU gradient**
Using $U3(1.44, 0.8, 2.1)$ as target gate

In [ ]:
def problem(wires):
  qml.U3(1.44, 0.8, 2.1, wires=wires)

In [691]:
def ansatz_circuit(ansatz, var, wires):
  var = padding(var)
  for i, gate in enumerate(ansatz):
    if var[i]:
      str2gate(gate)(var[i], wires=wires[i])  
    else:
      str2gate(gate)(wires=wires[i])  


@qml.qnode(dev1)
def cost_function(ansatz, params, wires):
  ansatz = [a.numpy() for a in ansatz]
  wires = [w.numpy() for w in wires]

  ansatz_circuit(ansatz[:-1], params, wires[:-1])

  problem(wires=wires[-1])

  return qml.expval(qml.Projector([1], wires=0))

@qml.qnode(dev1)
def prediction(ansatz, params, wires):
  ansatz = [a.numpy() for a in ansatz]
  wires = [w.numpy() for w in wires]

  ansatz_circuit(ansatz[:-1], params, wires[:-1])
  
  return qml.probs(wires=[0])

@qml.qnode(dev1)
def target(wires):
  problem(wires=wires[-1].numpy())
  return qml.probs(wires=[0])

In [692]:
np.random.seed(1)

num_layers = 2
num_wires = 1

ansatz = ['H', 'RX', 'RY', 'RZ']*num_layers + ['Problem']
wires = [0, 0, 0, 0]*num_layers + [0]


params_init= 0.05*np.random.randn(1, 3*num_wires*num_layers)[0]
params_init

tensor([ 0.08121727, -0.03058782, -0.02640859, -0.05364843,  0.04327038,
        -0.11507693], requires_grad=True)

In [693]:
print_ansatz(ansatz, params_init, wires)

 0: ──H──RX(0.0812)──RY(-0.0306)──RZ(-0.0264)──H──RX(-0.0536)──RY(0.0433)──RZ(-0.115)──Rot(2.1, 1.44, -2.1)──Rϕ(2.1)──Rϕ(0.8)──┤ State 



In [697]:
print("Initial cost: ", cost_function(ansatz, params_init, wires))

Initial cost:  0.38420647954293236


In [698]:
print("Initial prediction: ", prediction(ansatz, params_init, wires))

Initial prediction:  [0.99703768 0.00296232]


In [702]:
opt = GradientDescentOptimizer(0.01)

var = params_init.copy()
loss_plot = []

for it in range(1201):# while True:
    var, _cost = opt.step_and_cost(lambda v: cost_function(ansatz, v, wires), var, 
                                   grad_fn=lambda var: lcu_gradients(ansatz, H, ham_wires, ham_coeffs, var, wires)) 
    loss_plot.append(_cost)

    if it%100==0:
      print("Iter: {:5d} | Cost: {:0.11f} ".format(it, _cost))

Iter:     0 | Cost: 0.38420647954 
Iter:   100 | Cost: 0.07905926083 
Iter:   200 | Cost: 0.01260131677 
Iter:   300 | Cost: 0.00204853691 
Iter:   400 | Cost: 0.00034842518 
Iter:   500 | Cost: 0.00006112723 
Iter:   600 | Cost: 0.00001091990 
Iter:   700 | Cost: 0.00000197059 
Iter:   800 | Cost: 0.00000035761 
Iter:   900 | Cost: 0.00000006510 
Iter:  1000 | Cost: 0.00000001187 
Iter:  1100 | Cost: 0.00000000217 
Iter:  1200 | Cost: 0.00000000040 


In [709]:
#@title 
fig = px.line(pd.DataFrame({"Iteration":range(len(loss_plot)), "Loss":loss_plot}), 
                          x="Iteration", y="Loss", title="Discrete-variable LCU loss", width=1000)
fig.show()

In [708]:
print("Target    : ", target(wires))
print("Prediction: ", prediction(ansatz[:-1], var, wires[:-1]))

Target    :  [0.56521185 0.43478815]
Prediction:  [0.56522923 0.43477077]
